In [ ]:
import os
import random
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from keras.optimizers import Adam, SGD, Adagrad, RMSprop
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Conv2D , MaxPool2D, Flatten, Dense
from tensorflow.keras import datasets, layers, models
from sklearn.metrics import accuracy_score, confusion_matrix
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, Callback
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.models import Model
from matplotlib import pyplot
from sklearn.metrics import ConfusionMatrixDisplay
from numpy import expand_dims
from tensorflow.keras.preprocessing import image_dataset_from_directory
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

#### Reading and normalizing data

In [ ]:

train = image_dataset_from_directory(
    'Group_24/train/',
    labels='inferred',
    label_mode='categorical',
    batch_size=1,
    image_size=(224, 224),
    shuffle=True,
    seed=42,
    validation_split=0.0
)

val = image_dataset_from_directory(
    'Group_24/val/',
    labels='inferred',
    label_mode='categorical',
    batch_size=1,
    image_size=(224, 224),
    shuffle=True,
    seed=42,
    validation_split=0.0
)

test = image_dataset_from_directory(
    'Group_24/test/',
    labels='inferred',
    label_mode='categorical',
    batch_size=1,
    image_size=(224, 224),
    shuffle=True,
    seed=42,
    validation_split=0.0
)
class_names = train.class_names
def normalize(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image, label

train_data = train.map(normalize)
val_data = val.map(normalize)
test_data = test.map(normalize)

#### Arch 1

In [ ]:
input_layer = layers.Input(shape=(224, 224, 3))
conv1 = layers.Conv2D(8, (11, 11), strides=4, padding='same', activation='relu')(input_layer)
max_pool1 = layers.MaxPooling2D((3, 3), strides=(2, 2))(conv1)
conv2 = layers.Conv2D(16, (5, 5), strides=1, padding='same', activation='relu')(max_pool1)
max_pool2 = layers.MaxPooling2D((3, 3), strides=2)(conv2)
flatten = layers.Flatten()(max_pool2)
dense1 = layers.Dense(128, activation='relu')(flatten)
output_layer = layers.Dense(5, activation ='softmax')(dense1)

adam_optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
# model.compile(optimizer=adam_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model = models.Model(inputs=input_layer, outputs=output_layer)

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=0), optimizer=adam_optimizer, metrics=['accuracy'])

# callbacks
my_callbacks = [
    EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10),
    TensorBoard(log_dir=f'./logdir/Q1/Arch1/')
]
# out = model.fit(train_data, validation_data=val_data, epochs=100, callbacks=my_callbacks)
model_fit = model.fit(train_data,validation_data=val_data, batch_size=len(train_data), epochs=100, verbose=0, callbacks=my_callbacks, validation_split=0.0, shuffle=True, validation_batch_size=None)

hist_metric = 'accuracy'
print(f'epochs: {len(model_fit.history[hist_metric])}, acc: {model_fit.history[hist_metric][-1]}\n')
model.save(f'models/Q1/Arch1.tf')


Plotting accuracy

In [ ]:
# Plot accuracy history
acc = model_fit.history['accuracy']
val_acc = model_fit.history['val_accuracy']
epochs = range(1, len(acc) + 1)

plt.figure()
plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy (Arch 1)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

Plotting confusion matrix

In [ ]:
# train accuracy and confusion matrix
train_loss, train_acc = model.evaluate(train_data, verbose=0)
train_pred = model.predict(train_data)
train_labels = tf.concat([q for p, q in train_data], axis=0)
print("Confusion Matrix:")
print(tf.math.confusion_matrix(tf.argmax(train_labels,axis=1),tf.argmax(train_pred,axis=1),num_classes=5))
# val accuracy and confusion matrix
val_loss, val_acc = model.evaluate(val_data, verbose=0)
val_pred = model.predict(val_data)
val_labels = tf.concat([q for p, q in val_data], axis=0)
print("Confusion Matrix:")
print(tf.math.confusion_matrix(tf.argmax(val_labels,axis=1),tf.argmax(val_pred,axis=1),num_classes=5))

In [ ]:
train_loss, train_acc = model.evaluate(train_data, verbose=0)
train_pred = model.predict(train_data)
train_labels = tf.concat([q for p, q in train_data], axis=0)

val_loss, val_acc = model.evaluate(val_data, verbose=2)
val_pred = model.predict(val_data)
val_labels = tf.concat([q for p, q in val_data], axis=0)

print('Confusion matrix (train):')
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111)
cm_display = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(tf.argmax(train_labels,axis=1),tf.argmax(train_pred,axis=1)), display_labels=class_names)
cm_display.plot(ax = ax, cmap='Greys', colorbar=True)
plt.show()

print('Confusion matrix (val):')
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111)
cm_display = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(tf.argmax(val_labels,axis=1),tf.argmax(val_pred,axis=1)), display_labels=class_names)
cm_display.plot(ax = ax, cmap='Greys', colorbar=True)
plt.show()

#### Arch 2

In [ ]:
input_layer = layers.Input(shape=(224, 224, 3))
conv1 = layers.Conv2D(8, (11, 11), strides=4, padding='same', activation='relu')(input_layer)
max_pool1 = layers.MaxPooling2D((3, 3), strides=(2, 2))(conv1)
conv2 = layers.Conv2D(16, (5, 5), strides=1, padding='same', activation='relu')(max_pool1)
max_pool2 = layers.MaxPooling2D((3, 3), strides=2)(conv2)
conv3 = layers.Conv2D(32, (3, 3), strides=1, padding='same', activation='relu')(max_pool2)
max_pool3 = layers.MaxPooling2D((3,3), strides=2)(conv3)
flatten = layers.Flatten()(max_pool3)
dense1 = layers.Dense(128, activation='relu')(flatten)
output_layer = layers.Dense(5, activation ='softmax')(dense1)

adam_optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
# model.compile(optimizer=adam_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model = models.Model(inputs=input_layer, outputs=output_layer)

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=0), optimizer=adam_optimizer, metrics=['accuracy'])

# callbacks
my_callbacks = [
    EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10),
    TensorBoard(log_dir=f'./logdir/Q1/Arch2/')
]
# out = model.fit(train_data, validation_data=val_data, epochs=100, callbacks=my_callbacks)
model_fit = model.fit(train_data,validation_data=val_data, batch_size=len(train_data), epochs=100, verbose=0, callbacks=my_callbacks, validation_split=0.0, shuffle=True, validation_batch_size=None)

hist_metric = 'accuracy'
print(f'epochs: {len(model_fit.history[hist_metric])}, acc: {model_fit.history[hist_metric][-1]}\n')
model.save(f'models/Q1/Arch2.tf')


Plotting accuracy

In [ ]:
# Plot accuracy history
acc = model_fit.history['accuracy']
val_acc = model_fit.history['val_accuracy']
epochs = range(1, len(acc) + 1)

plt.figure()
plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy (Arch 1)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

Plotting confusion matrix

In [ ]:
# train accuracy and confusion matrix
train_loss, train_acc = model.evaluate(train_data, verbose=0)
train_pred = model.predict(train_data)
train_labels = tf.concat([q for p, q in train_data], axis=0)
print("Confusion Matrix:")
print(tf.math.confusion_matrix(tf.argmax(train_labels,axis=1),tf.argmax(train_pred,axis=1),num_classes=5))
# val accuracy and confusion matrix
val_loss, val_acc = model.evaluate(val_data, verbose=0)
val_pred = model.predict(val_data)
val_labels = tf.concat([q for p, q in val_data], axis=0)
print("Confusion Matrix:")
print(tf.math.confusion_matrix(tf.argmax(val_labels,axis=1),tf.argmax(val_pred,axis=1),num_classes=5))

In [ ]:
train_loss, train_acc = model.evaluate(train_data, verbose=0)
train_pred = model.predict(train_data)
train_labels = tf.concat([q for p, q in train_data], axis=0)

val_loss, val_acc = model.evaluate(val_data, verbose=2)
val_pred = model.predict(val_data)
val_labels = tf.concat([q for p, q in val_data], axis=0)

print('Confusion matrix (train):')
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111)
cm_display = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(tf.argmax(train_labels,axis=1),tf.argmax(train_pred,axis=1)), display_labels=class_names)
cm_display.plot(ax = ax, cmap='Greys', colorbar=True)
plt.show()

print('Confusion matrix (val):')
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111)
cm_display = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(tf.argmax(val_labels,axis=1),tf.argmax(val_pred,axis=1)), display_labels=class_names)
cm_display.plot(ax = ax, cmap='Greys', colorbar=True)
plt.show()

#### Arch 3

In [ ]:
input_layer = layers.Input(shape=(224, 224, 3))
conv1 = layers.Conv2D(8, (11, 11), strides=4, padding='same', activation='relu')(input_layer)
max_pool1 = layers.MaxPooling2D((3, 3), strides=(2, 2))(conv1)
conv2 = layers.Conv2D(16, (5, 5), strides=1, padding='same', activation='relu')(max_pool1)
max_pool2 = layers.MaxPooling2D((3, 3), strides=2)(conv2)
conv3 = layers.Conv2D(32, (3, 3), strides=1, padding='same', activation='relu')(max_pool2)
conv4 = layers.Conv2D(64, (3, 3), strides=1, padding='same', activation='relu')(conv3)
max_pool3 = layers.MaxPooling2D((3, 3), strides=2)(conv4)
flatten = layers.Flatten()(max_pool3)
dense1 = layers.Dense(128, activation='relu')(flatten)
output_layer = layers.Dense(5, activation ='softmax')(dense1)

adam_optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
# model.compile(optimizer=adam_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model = models.Model(inputs=input_layer, outputs=output_layer)

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=0), optimizer=adam_optimizer, metrics=['accuracy'])

# callbacks
my_callbacks = [
    EarlyStopping(monitor='val_loss', min_delta=1e-7, patience=10),
    TensorBoard(log_dir=f'./logdir/Q1/Arch3/')
]
# out = model.fit(train_data, validation_data=val_data, epochs=100, callbacks=my_callbacks)
model_fit = model.fit(train_data,validation_data=val_data, batch_size=len(train_data), epochs=100, verbose=0, callbacks=my_callbacks, validation_split=0.0, shuffle=True, validation_batch_size=None)

hist_metric = 'accuracy'
print(f'epochs: {len(model_fit.history[hist_metric])}, acc: {model_fit.history[hist_metric][-1]}\n')
model.save(f'models/Q1/Arch3.tf')


Plotting accuracy

In [ ]:
# Plot accuracy history
acc = model_fit.history['accuracy']
val_acc = model_fit.history['val_accuracy']
epochs = range(1, len(acc) + 1)

plt.figure()
plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy (Arch 1)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

Plotting confusion matrix

In [ ]:
# train accuracy and confusion matrix
train_loss, train_acc = model.evaluate(train_data, verbose=0)
train_pred = model.predict(train_data)
train_labels = tf.concat([q for p, q in train_data], axis=0)
print("Confusion Matrix:")
print(tf.math.confusion_matrix(tf.argmax(train_labels,axis=1),tf.argmax(train_pred,axis=1),num_classes=5))
# val accuracy and confusion matrix
val_loss, val_acc = model.evaluate(val_data, verbose=0)
val_pred = model.predict(val_data)
val_labels = tf.concat([q for p, q in val_data], axis=0)
print("Confusion Matrix:")
print(tf.math.confusion_matrix(tf.argmax(val_labels,axis=1),tf.argmax(val_pred,axis=1),num_classes=5))

In [ ]:
train_loss, train_acc = model.evaluate(train_data, verbose=0)
train_pred = model.predict(train_data)
train_labels = tf.concat([q for p, q in train_data], axis=0)

val_loss, val_acc = model.evaluate(val_data, verbose=2)
val_pred = model.predict(val_data)
val_labels = tf.concat([q for p, q in val_data], axis=0)

print('Confusion matrix (train):')
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111)
cm_display = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(tf.argmax(train_labels,axis=1),tf.argmax(train_pred,axis=1)), display_labels=class_names)
cm_display.plot(ax = ax, cmap='Greys', colorbar=True)
plt.show()

print('Confusion matrix (val):')
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111)
cm_display = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(tf.argmax(val_labels,axis=1),tf.argmax(val_pred,axis=1)), display_labels=class_names)
cm_display.plot(ax = ax, cmap='Greys', colorbar=True)
plt.show()

#### Feature maps for best architecture

In [ ]:
image = [image.numpy() for image, label in train_data.take(1)][0]
plt.imshow(image[0])
plt.axis(False)
plt.show()

In [ ]:
# feature maps
for i in [1,3,5]:
    part_model = Model(inputs=model.inputs, outputs=model.layers[i].output)
    # get feature map for first hidden layer
    print(image.shape)
    feature_maps = part_model.predict(image)

    fig, ax = plt.subplots(2,4, figsize=(8,4))
    for j in range(8):
        ax[int(j/4)][int(j%4)].imshow(feature_maps[0, :, :, j],'viridis')
        ax[int(j/4)][int(j%4)].axis("off")
    plt.axis("off")
    plt.show()

#### Maximally activating patch

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Load your trained model
model = tf.keras.models.load_model('your_model.h5')

# Select a sample image from the training set of each class
class_1_image = ...
class_2_image = ...
class_3_image = ...
class_4_image = ...
class_5_image = ...

# Get the output of the last convolutional layer
last_conv_layer = model.get_layer('name_of_last_conv_layer')
last_conv_layer_model = tf.keras.Model(model.inputs, last_conv_layer.output)

# Get the gradients of the output of the last convolutional layer with respect to the input image
grad_model = tf.keras.models.Model([model.inputs], tf.gradients(last_conv_layer_model.output, model.inputs))

# Define a function to get the most activated patch in the input image
def get_most_activated_patch(img, model):
    # Reshape the input image to match the model's input shape
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = np.expand_dims(img, axis=0)

    # Get the output of the last convolutional layer and the gradients of the output with respect to the input
    last_conv_output = last_conv_layer_model.predict(img)
    grads = grad_model(img)

    # Get the most activated neuron in the last convolutional layer
    max_activation_idx = np.argmax(last_conv_output[0,:,:,0])
    max_activation = last_conv_output[0,:,:,max_activation_idx]

    # Get the gradients corresponding to the most activated neuron
    pooled_grads = np.mean(grads[0], axis=(0, 1, 2))
    activation_grads = last_conv_output[0,:,:,max_activation_idx] * pooled_grads

    # Get the most activated patch in the input image
    heatmap = np.mean(activation_grads, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    heatmap = heatmap.astype('float32')
    img = img[0,:,:,:]
    heatmap = tf.image.resize(heatmap, (img.shape[0], img.shape[1]))
    heatmap = heatmap.numpy()
    mask = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(img, 0.5, heatmap, 0.5, 0)
    return superimposed_img

# Get the most activated patch in each image and display them
plt.subplot(1, 5, 1)
plt.imshow(get_most_activated_patch(class_1_image, model))
plt.subplot(1, 5, 2)
plt.imshow(get_most_activated_patch(class_2_image, model))
plt.subplot(1, 5, 3)
plt.imshow(get_most_activated_patch(class_3_image, model))
plt.subplot(1, 5, 4)
plt.imshow(get_most_activated_patch(class_4_image, model))
plt.subplot(1, 5, 5)
plt.imshow(get_most_activated_patch(class_5_image, model))
plt.show()
